<div class='status' style='background-color: #183a4b; color: white; padding-top: 4px; padding-bottom: 4px; padding-left: 20px; padding-right: 20px; border-radius: 10px; font-family: Arial, sans-serif; font-size: 26px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.5);'><b>Fake GP Reviews - FAKE Reviews SCORER</div>

## Loading Libraries

In [6]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import datetime
import os
from tqdm import tqdm

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
warnings.filterwarnings("ignore")

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

<div class='status' style='background-color: #f5c244; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'><b>Loading</b> dataset</div>

In [8]:
fake_data = pd.read_csv("../ai_medreview/data/fake_gp_reviews_01_processed.csv")
fake_data.head(2)

,free_text,generation_category,free_text_len,sentiment_free_text,sentiment_score_free_text,feedback_labels,emotion_free_text
0,The GP practice is run by a friendly woman who...,Patient Respect,878,positive,0.873504,Reception Staff Interaction,disappointment
1,Staff were very friendly but slow. \n\nThe GP ...,Patient Respect,88,positive,0.441180,Reception Staff Interaction,disappointment


In [ ]:
fake_data.columns

In [ ]:
fake_data['label'] = 1

In [ ]:
fake_data['abs_sentiment'] = fake_data.apply(
    lambda row: -row['sentiment_score_free_text'] if row['sentiment_free_text'] == 'negative' else row['sentiment_score_free_text'],
    axis=1
)

In [ ]:
fake_data.head(20)

# Scorer

In [5]:
!pip install tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [10]:
from jan883_codebase.llm.ollama import ask_ollama

In [14]:
score_list = []

for review in tqdm(fake_data['free_text'], desc=f"Scoring: ", total=fake_data.shape[0]):
    prompt = f"""Your task is to score the GP Surgery on the question:
“Overall, how was your experience of our service?”

You have already written your review:

{review}

Scoring Criteria (0 - 5 scale):
	•	5 = Very good
	•	4 = Good
	•	3 = Neither good nor poor
	•	2 = Poor
	•	1 = Very poor
	•	0 = Don’t know

🔹 How to score:
	•	If your review is highly positive, confidently score 5 or 4.
	•	If your review is mixed or neutral, use 3 appropriately.
	•	If your review is negative, score 2 or 1, depending on the severity of the criticism.
	•	If the review expresses uncertainty or lacks clarity, consider 0 (Don’t know).

🛑 Output only the score as an INTEGER (0-5). Do NOT include any explanations, text, or comments.
🎯 Be decisive with your score—let it reflect the tone of the review.

⚠️ Remember: This is a FAKE GP Surgery review. Your score should reflect the sentiment in the review realistically."""
    
    output = ask_ollama(
        prompt,
        system_prompt='You are a patient at a GP Surgery who is an expert at writing fake reviews.',
        model='nezahatkorkmaz/deepseek-v3:latest',
        temp=1.3,
    )
    print(output, end='\r')
    score_list.append(output)


Scoring:   0%|                                                  | 1/1935 [00:03<1:50:52,  3.44s/it]

4

Scoring:   0%|                                                    | 2/1935 [00:04<58:30,  1.82s/it]

4

Scoring:   0%|                                                    | 3/1935 [00:05<50:57,  1.58s/it]

...

4

Scoring: 100%|██████████████████████████████████████████████▉| 1934/1935 [1:40:59<00:01,  1.50s/it]

3

Scoring: 100%|███████████████████████████████████████████████| 1935/1935 [1:41:00<00:00,  3.13s/it]

3

In [15]:
# Final save, to ensure we capture any remaining reviews 
fake_data['rating'] = score_list

In [16]:
fake_data.to_csv('../ai_medreview/data/fake_gp_reviews_01_processed.csv', index=False)